In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 148.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


## Local Inference on GPU
Model page: https://huggingface.co/context-labs/meta-llama-Llama-3.2-3B-Instruct-FP16

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/context-labs/meta-llama-Llama-3.2-3B-Instruct-FP16)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving PV_Signal_List_10_TEST_SCENARIOS.xlsx to PV_Signal_List_10_TEST_SCENARIOS.xlsx


In [ ]:
import pandas as pd
df = pd.read_excel("PV_Signal_List_10_TEST_SCENARIOS.xlsx")
print(df.head())


       Signal term Date detected                                     Status  \
0           Stroke        May-25                                    Ongoing   
1              SJS        Jun-25                                    Refuted   
2      Myocarditis        Apr-25   Closed as Potential Risk (Not Important)   
3  QT prolongation        Sep-24       Closed as Potential Risk (Important)   
4     Pancreatitis        Mar-25  Closed as Identified Risk (Not Important)   

  Date closed     Source of signal  \
0         NaN        Meta-analysis   
1      25-Aug         Case reports   
2      10-Jul             Registry   
3      12-Oct       Clinical Study   
4      09-Sep  Spontaneous Reports   

                                          Summary              Action  
0                   Signal still under assessment             Pending  
1          Evaluation shows no causal association                 NaN  
2  Evidence suggests possible risk but low impact             Monitor  
3       

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "context-labs/meta-llama-Llama-3.2-3B-Instruct-FP16"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import time
system_prompt = (
    "You are an expert in pharmacovigilance and ICH E2C(R2) guidelines. "
    "Your task is to read ONLY the input safety signal data exactly as provided "
    "and map each signal to the correct PBRER section(s) according to Appendix F. "
    "Do NOT infer medical judgment beyond what is explicitly stated.\n\n"

    "Use ONLY the following seven scenarios when categorizing signals:\n"
    "1. Ongoing Signal\n"
    "2. Closed + Refuted\n"
    "3. Closed → Potential Risk (NOT important)\n"
    "4. Closed → Potential Risk (IMPORTANT)\n"
    "5. Closed → Identified Risk (NOT important)\n"
    "6. Closed → Identified Risk (IMPORTANT)\n"
    "7. Closed → New Information on Known Risk\n\n"

    "Mapping logic:\n"
    "- Ongoing Signal → Section 15 only.\n"
    "- Closed + Refuted → Section 15 + Section 16.2.\n"
    "- Closed → Potential Risk (NOT important) → Section 15 + Section 16.2.\n"
    "- Closed → Potential Risk (IMPORTANT) → Section 15 + Section 16.2 + Section 16.4.\n"
    "- Closed → Identified Risk (NOT important) → Section 15 + Section 16.2.\n"
    "- Closed → Identified Risk (IMPORTANT) → Section 15 + Section 16.2 + Section 16.4.\n"
    "- Closed → New Information on Known Risk → Section 15 + Section 16.3 + Section 16.4.\n\n"

    "Very important rules:\n"
    "- If the dataset does NOT explicitly state identified/potential/important, do NOT infer it.\n"
    "- Do NOT perform causality assessment.\n"
    "- If status wording is unclear, categorize as Ongoing or Refuted only.\n"
    "- You MUST strictly restrict your reasoning to the information given in the dataset.\n"
    "- Do NOT hallucinate: do NOT invent risks, actions, importance level, severity, "
    "additional evidence, or clinical interpretations not present in the input.\n"
    "- Do NOT add assumptions or extend the summary; use ONLY what is written in the row.\n\n"

    "OUTPUT FORMAT REQUIREMENT (MANDATORY):\n"
    "Your response MUST contain exactly two lines of text:\n"
    "Mapped Sections: <comma-separated section names>\n"
    "Decision Logic: <short explanation referencing ONLY the given dataset and rule table>\n\n"

    "Example expected output:\n"
    "Mapped Sections: Section 15\n"
    "Decision Logic: Status indicates the signal is still under evaluation. "
    "According to ICH E2C(R2), ongoing signals are only listed in Section 15.\n\n"

    "JSON OUTPUT FORMAT (in addition to the two-line output):\n"
    "{\n"
    "  \"signal\": \"<signal term>\",\n"
    "  \"scenario\": \"<one of the seven scenarios>\",\n"
    "  \"mapped_sections\": [\"Section 15\", \"Section 16.2\", ...],\n"
    "  \"categorization_summary\": \"One-sentence classification summary. Must use ONLY dataset information.\",\n"
    "  \"decision_logic\": \"Full reasoning using rule-based logic; must NOT hallucinate or infer beyond input.\"\n"
    "}\n\n"

    "Now read the input row and generate BOTH the two-line output AND the JSON output."
)

results = []

for _, row in df.iterrows():
    signal_info = (
        f"Signal: {row['Signal term']}\n"
        f"Date detected: {row['Date detected']}\n"
        f"Status: {row['Status']}\n"
        f"Date closed: {row['Date closed']}\n"
        f"Source of signal: {row['Source of signal']}\n"
        f"Summary: {row['Summary']}\n"
        f"Action: {row['Action']}"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"signal_info:\n{signal_info}"}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    input_token_count = inputs["input_ids"].shape[-1]
    start_time = time.time()

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.4
    )

    end_time = time.time()
    gen_time = round(end_time - start_time, 2)

    generated_ids = outputs[0][input_token_count:]
    generated_token_count = generated_ids.shape[-1]

    tokens_per_sec = round(generated_token_count / gen_time, 2)

    response = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()

    row_result = row.to_dict()
    row_result["Mapping"] = response
    row_result["Gen Time"] = gen_time
    row_result["Tokens Per Sec"] = tokens_per_sec

    results.append(row_result)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
from google.colab import files
# Save results
mapped_df = pd.DataFrame(results)
mapped_df.to_excel("mapped_signals_llama.xlsx", index=False)
files.download("mapped_signals_llama.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>